# **ANÁLISE, TRATAMENTO, AUTOMATIZAÇÃO E MACHINE LEARNING: GEOLOCALIZAÇÃO REGIÃO**


## **Objetivo**

- Localizar a região de cada pessoa com suas coordenadas dentro de um munícipio na utilização de funções relacionadas a dados geoespacias;
- A partir de localizada, construir um modelo de aprendizagem de máquina que realizará o mesmo próposito de prever a região;
- Separar os dados em uma amostra, usada na criação do modelo, e o restante para previsão da região pelo modelo;
- Empregar quatro variáveis preditivas criadas na conversão de latitude e longitude, destaque na utilização de algoritmo de clusterização agrupamento hierárquico;
- Testar em diferentes algoritmos de aprendizagem supervisionada de classificação e comparar o melhor para fazer a previsão da região nos dados que restaram.  


## **Informação**  


Base de dados: 185.005 linhas com 20% para elaborar o modelo de machine learning e os outros 80% para visualizar o desempenho do próprio modelo na rotulação do atributo-alvo região.

## **INTRODUÇÃO**

### **Conceito dados geoespaciais utilizados**

#### **Ponto**

Uma coordenada definida por latitude e longitude. Por exemplo a localização das pessoas que serão usadas no projeto.  

![Ponto](https://user-images.githubusercontent.com/111579476/196970550-1d1b62df-48e5-4370-9f1c-dea9a4d33fc7.png)  

Disponível em: https://www.slideshare.net/MdYousufGazi/introductory-gis

#### **Polígono**

São os pontos interligados com uma área delimitada por dentro, ou seja, fechados por dentro. Cada região que compõem o município pode ser expressa por um polígono e será essencial no objetivo do trabalho.

![Poligono](https://user-images.githubusercontent.com/111579476/196970893-096e611b-33f7-4887-8506-730b0ba17b9c.png)
 
Disponível em: https://www.slideshare.net/MdYousufGazi/introductory-gis 

#### **Line String**

São os pontos interligados. No arquivo GeoJSON representa a cidade.

![Line_String](https://user-images.githubusercontent.com/111579476/196971610-e524a0e8-767e-4d19-bc97-d542e70c0de2.png)  
Disponível em: http://132.72.155.230:3838/js/geojson-1.html

### **GIS (Sistemas de Informação Geográfica)**

" Um sistema de informação geográfica (GIS) é um sistema que cria, gerencia, analisa e mapeia todos os tipos de dados. GIS conecta dados a um mapa, integrando dados de localização (onde as coisas estão) com todos os tipos de informações descritivas (como as coisas são lá). Isso fornece uma base para mapeamento e análise que é usada na ciência e em quase todos os setores. O GIS ajuda os usuários a entender padrões, relacionamentos e contexto geográfico. Os benefícios incluem melhor comunicação e eficiência, bem como melhor gestão e tomada de decisões."   
Disponível em: https://www.esri.com/pt-br/what-is-gis/overview

Utilizou-se o conceito de Intersecção presente na nomenclatura do GIS que se baseia na entrada de tipos de geometria (ponto, polígonos, linhas) e a saída resulta na sobreposição desses objetos espacial, no contexto do projeto verificou se os pontos geográficos das pessoas estão dentro dos limites de um poígono da região A, B, C e assim por diante.  

<img width="355" alt="Intersect" src="https://user-images.githubusercontent.com/111579476/196972254-69472111-cf51-405e-92a5-b81eb1f6d9fa.png">  


Disponível em: https://pro.arcgis.com/en/pro-app/latest/tool-reference/analysis/intersect.htm

## **ETAPA INICIAL**

### **Análise**

In [ ]:
# Definição da biblioteca
import os 
import json
import pandas as pd
import logging
from pandas.io.json import json_normalize
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.geometry import Point
import shapely.speedups as sp

In [ ]:
# Ver o diretório atual
diretorio = os.getcwd()


In [ ]:
# Ler o arquivo com as coordenadas Lat e Long de uma população em um município
diretorio = os.path.abspath(r'../') # retornar um diretório
dataset_principal = pd.read_csv(diretorio + "\Dataset\Id_Coord.csv", sep = ",", encoding = "UTF-8" )

In [ ]:
# Cabeçalho do dataset
dataset_principal.head()

In [ ]:
# Últimas linhas
dataset_principal.tail()

In [ ]:
# Verificar os tipos das variáveis
dataset_principal.dtypes

In [ ]:
# carregar o arquivo em formato geojson, usa-se para ler como json
with open(diretorio + '\Shapes\Regiao.geojson', encoding= 'UTF-8') as f:
        datajson = json.load(f)

In [ ]:
# converter em dataframe baseado na estrutura desse arquivo
dataset_geojson_regiao = json_normalize(datajson["features"])

In [ ]:
dataset_geojson_regiao.head()

In [ ]:
# Dimensionalidade dos dados (linha, coluna)
dataset_geojson_regiao.shape

### **Tratamento**

In [ ]:
# Percebe-se que o dataset do geojson precisa ser tratado e consequetemente extrair as coordenadas da latitude e longitude presente em um polígono
# Nesse caso, se quer trabalhar com polígono e não a cidade do tipo LineString.
# Obs: Se for somente polígonos não há necessidade dessa filtragem
data_geojson_regiao = dataset_geojson_regiao.loc[dataset_geojson_regiao["geometry.type"] == "Polygon", ["geometry.coordinates", "properties.name"]]
 

In [ ]:
# Extração das coordenadas da coluna geometry.coordinates
# Primeiro passo retirar uma série de colchetes que é responsável por formar o polígono
# Cada linha do dataset representa um ponto de uma tal região com coordenadas
dataset_map = (data_geojson_regiao['geometry.coordinates'].apply(lambda r: [(coor[0],coor[1]) for coor in r[0]])
        .apply(pd.Series).stack()
        .reset_index(level=1).rename(columns={0:'geometry.coordinates',"level_1":"point"})
        .join(data_geojson_regiao.drop('geometry.coordinates',1), how='left')).reset_index(level=0)

In [ ]:
dataset_map

In [ ]:
dataset_map.shape

In [ ]:
# Obter a latitude e longitude
dataset_map[['long','lat']] = dataset_map['geometry.coordinates'].apply(pd.Series)
dataset_map


In [ ]:
# Excluir as colunas irrelevantes como index, point e claro geometry.coordinates que já se trabalhou
dataset_map = dataset_map.drop(dataset_map.columns[[0,1, 2]],axis = 1) # index das colunas
dataset_map


**Resumo**:  
- No dataset do arquivo geojson que se carregou veio para cada linha um polígono que é formado por varíos pontos da coluna geometry.coordinates;
- Assim se fez um tratamento para o dataset ficasse para cada linha fosse um ponto de uma tal região e não um polígono formado por uma série de valores encontrados no atributo geometry.coordinates;
- Como resultado se obteve a latitude e longitude dos vários pontos da região. Dessa forma possibilita no desenvolvimento da etapa de automatização.

## **AUTOMATIZAÇÃO**

In [ ]:
# Função que retornará o dataset que se quer saber a região onde reside
def geolocalizacao_get(dataset, dataset_map, id_unico):
    dataset_det_regiao = pd.DataFrame() # o dataset final que será retornado no final
    # Loop for que terá o valor de uma região, ou seja, passará por todas presentes no dataset_map
    for name_regiao in  sorted (dataset_map["properties.name"].unique()):
        dataset_each_regiao = dataset_map.loc[dataset_map["properties.name"] == name_regiao] # filtrar por região o dataset
        # Encontrar coordenadas das pessoas próximas da região
        PositionScale = dataset.index[(dataset.Long >= min(dataset_each_regiao.long)) &                     # oeste
            (dataset.Long <= max(dataset_each_regiao.long)) &                                               # leste
            (dataset.Lat <= max(dataset_each_regiao.lat)) &                                                 # norte
            (dataset.Lat >= min(dataset_each_regiao.lat))].tolist()                                         # sul
        Position_Scale_P = dataset.iloc[PositionScale]  
        if(Position_Scale_P.shape[0] == 0): # se nenhuma foi encontrado
            continue
        # Converter os dois datasets como objeto geodataframe
        Polygon_Regiao = gpd.GeoDataFrame(index = [0], crs = 'epsg:4326', geometry = [Polygon(zip(dataset_each_regiao["lat"], dataset_each_regiao["long"]))])
        Point_Coord = gpd.GeoDataFrame(Position_Scale_P, geometry = gpd.points_from_xy(Position_Scale_P["Lat"], Position_Scale_P["Long"])) # conversão para geodataframe na  localização
        sp.enable() # consulta mais rápida
        Geolocalizacao = Point_Coord.within(Polygon_Regiao.loc[0, 'geometry']) # verifcar os pontos retidos no polígono
        if(not any(Geolocalizacao)): # se nenhuma coordenada foi localizada em uma tal região 
            continue
        Position_Scale_P_Loc = Position_Scale_P.loc[Geolocalizacao]
        Position_Scale_P_Loc["Regiao"] = name_regiao # criar coluna região e colocar o nome da região
        # Adicionar o dataset com a região localizada no dataset final
        dataset_det_regiao = dataset_det_regiao.append(pd.DataFrame(data = Position_Scale_P_Loc[Position_Scale_P_Loc.columns.difference(['geometry'])]), ignore_index=True)
    dataset_coord_not_find = dataset[~dataset[id_unico].isin(dataset_det_regiao[id_unico])] # consulta para saber quem não foi localizado em uma região
    dataset_coord_not_find["Regiao"] = "Outros Municípios" # Colocar como Outro Município
    # Finalmente juntar como um só, assim se preserva com a mesma quantidade de linha e a coluna adicional região
    dataset_det_regiao = pd.concat([dataset_det_regiao, dataset_coord_not_find])
    return dataset_det_regiao

In [ ]:
dataset_final = geolocalizacao_get(dataset_principal, dataset_map, "CodMunicipe")


In [ ]:
dataset_final

## **MACHINE LEARNING**

### **Pré-Processamento**

In [ ]:
# Fazer uma cópia e retirar Outros Municípios
dataset_final2 = dataset_final.loc[dataset_final.Regiao != "Outros Municípios"]

In [ ]:
dataset_final2

Amostragem

In [ ]:
dataset_final2["Regiao"].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
# 20% dos dados originais
x_treino, x_teste, y_treino, amostra_estra = train_test_split(dataset_final2.drop('Regiao', axis = 1), dataset_final2['Regiao'], stratify = dataset_final2['Regiao'], test_size = 0.2)

In [ ]:
amostra_estra

In [ ]:
amostra_estra.value_counts()

In [ ]:
# criar um indice
amostra_estra = amostra_estra.index

In [ ]:
# coloca no dataset a amostra escolhida
dataset_final_amostra = dataset_final2.loc[amostra_estra, :]

dataset_final_amostra

In [ ]:
# o restante que é a população 
dataset_final2_res= dataset_final2.loc[~dataset_final2.index.isin(amostra_estra), :]
dataset_final2_res

Extrair X, Y e Z

Para o modelo funcionar é necessário deixar os dados da latitude e longitude numa escala próxima a zero, ou seja, uma normalização e também representar em três dimensões com uso de seno e cosseno.

In [ ]:
import numpy as np
dataset_final_amostra['x'] = np.cos(dataset_final_amostra['Lat']) * np.cos(dataset_final_amostra['Long'])
dataset_final_amostra['y'] = np.cos(dataset_final_amostra['Lat']) * np.sin(dataset_final_amostra['Long'])
dataset_final_amostra['z'] = np.sin(dataset_final_amostra['Lat'])
dataset_final_amostra

In [ ]:
dataset_final_amostra[["x", "y", "z"]].describe()

### **Clusterização**

In [ ]:
len(dataset_final_amostra["Regiao"].unique())

In [ ]:
# Fazer pela clusterização hieráquica, k-means ou DBSCAN. O melhor é AgglomerativeClustering
from sklearn.cluster import AgglomerativeClustering, KMeans, DBSCAN
agc = AgglomerativeClustering(n_clusters = 19, affinity='euclidean', linkage='ward')
dataset_final_amostra['cluster'] = agc.fit_predict(dataset_final_amostra[['Lat','Long']])
# kmeans = KMeans(19)
# clusters = kmeans.fit_predict(dataset_final_amostra[['Lat','Long']])
# dataset_final_amostra['cluster'] = kmeans.predict(dataset_final_amostra[['Lat','Long']])
# DBSCAN:
# dataset_final_amostra['cluster'] = DBSCAN(eps = 0.005, min_samples = 4).fit_predict(dataset_final_amostra[['Lat','Long']])

In [ ]:
dataset_final_amostra

In [ ]:
sorted (dataset_final_amostra["cluster"].unique())

In [ ]:

regiao_map = gpd.read_file(diretorio + '\Shapes\Regiao.geojson')
regiao_map


In [ ]:
# Olhar a distribuição do cluster nas regiões
import matplotlib.pyplot as plt 
import seaborn as sns
fig, ax = plt.subplots(figsize = (12, 10))
for x, dataset in dataset_final_amostra.groupby("cluster"):
    ax = sns.scatterplot(x= dataset["Long"], y= dataset["Lat"], label="Cluster {:g}".format(x), data=dataset) 
regiao_map.plot(ax=ax, facecolor="none", edgecolor= "blue");
plt.axis('off')
h_legend = plt.legend(ncol=2)   
# Ajustar a legenda horizontalmente 
renderer = plt.gcf().canvas.get_renderer()
y_shift = -0.2*h_legend.texts[0].get_window_extent(renderer).height
h_legend.texts[1].set_position((0, y_shift))
plt.tight_layout();

### **Atributos Previsores e Alvo**

In [ ]:
dataset_final_amostra

In [ ]:
dataset_final_amostra.dtypes

In [ ]:
previsores = dataset_final_amostra.iloc[:, 4:8].values
previsores

In [ ]:
previsores.shape

In [ ]:
alvo = dataset_final_amostra.iloc[:, 3].values
alvo

In [ ]:
alvo.shape

#### **Codificação de variável categórica (Alvo)**

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
le.fit(alvo)
alvo_cond = le.transform(alvo)


### **Previsores e Alvos da Amostra em Arquivos Disponíveis** 

In [ ]:

"""previsores = pd.read_csv(diretorio + '/Variaveis/previsores_amostra.csv',header=None)
previsores = previsores.values
previsores"""

In [ ]:
"""alvo_cond = pd.read_csv(diretorio + '/Variaveis/alvo_amostra.csv',header=None)
alvo_cond = alvo_cond.iloc[:, 0].values
alvo_cond"""

### **Base de Treino e Teste**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# x: previsores e y: alvo
x_treino, x_teste, y_treino, y_teste = train_test_split(previsores, alvo_cond, 
                                                           stratify = alvo_cond, test_size= 0.30)

In [ ]:
x_treino.shape

In [ ]:
x_treino

In [ ]:
x_teste.shape

In [ ]:
x_teste

In [ ]:
y_treino.shape

In [ ]:
y_treino

In [ ]:
y_teste.shape

In [ ]:
y_teste

### **Máquina de Vetores de Suporte (SVM)**

In [ ]:
from sklearn.svm import SVC


In [ ]:
svm = SVC(kernel = 'rbf', random_state = 1, C = 50)
svm.fit(x_treino, y_treino)

In [ ]:
previsoes_svm = svm.predict(x_teste)
previsoes_svm

In [ ]:
y_teste

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
print(" Acurácia: %.2f%%" % (accuracy_score(y_teste, previsoes_svm) * 100.0))

In [ ]:
# le.inverse_transform(): volta a variável decodificada
pd.crosstab(le.inverse_transform(y_teste), le.inverse_transform(previsoes_svm), rownames=['Atual'], colnames=['Previsto'], margins=True)

In [ ]:
print(classification_report(le.inverse_transform(y_teste), le.inverse_transform(previsoes_svm)))

#### **Análise dados de treino**

In [ ]:
previsoes_treino = svm.predict(x_treino)
previsoes_treino

In [ ]:
print(" Acurácia: %.2f%%" % (accuracy_score(y_treino, previsoes_treino) * 100.0))

In [ ]:
pd.crosstab(le.inverse_transform(y_treino), le.inverse_transform(previsoes_treino), rownames=['Atual'], colnames=['Previsto'], margins=True)

In [ ]:
print(classification_report(le.inverse_transform(y_treino), le.inverse_transform(previsoes_treino)))

#### **Validação Cruzada**

In [ ]:
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score

In [ ]:
# separando os dados em folds
kfold = KFold(n_splits = 30, shuffle = True, random_state = 5)

In [ ]:
# Criando o modelo
modelo = SVC(kernel = 'rbf', random_state = 1, C = 50 )
resultado = cross_val_score(modelo, previsores, alvo_cond, cv = kfold)

# usamos a média e o desvio padrão
print(" Acurácia média: %.2f%%" % (resultado.mean() * 100.0))

### **Árvore de Decisão**

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
arvore = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
arvore.fit(x_treino, y_treino)

In [ ]:
previsoes_arvore = arvore.predict(x_teste)
previsoes_arvore

In [ ]:
y_teste

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
print(" Acurácia: %.2f%%" % (accuracy_score(y_teste, previsoes_arvore) * 100.0))

In [ ]:
pd.crosstab(le.inverse_transform(y_teste), le.inverse_transform(previsoes_arvore), rownames=['Atual'], colnames=['Previsto'], margins=True)

In [ ]:
print(classification_report(le.inverse_transform(y_teste), le.inverse_transform(previsoes_arvore)))

#### **Análise dados de treino**

In [ ]:
previsoes_treino = arvore.predict(x_treino)
previsoes_treino

In [ ]:
accuracy_score(y_treino, previsoes_treino)

In [ ]:
pd.crosstab(le.inverse_transform(y_treino), le.inverse_transform(previsoes_treino), rownames=['Atual'], colnames=['Previsto'], margins=True)

#### **Validação Cruzada**

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [ ]:
# Separando os dados em folds
kfold = KFold(n_splits = 30, shuffle=True, random_state = 5)

In [ ]:
modelo = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
resultado = cross_val_score(modelo, previsores, alvo_cond, cv = kfold)
# Usamos a média e o desvio padrão
print("Acurácia Média: %.2f%%" % (resultado.mean() * 100.0))

## **Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
random = RandomForestClassifier(n_estimators = 200, criterion = 'entropy', random_state = 0)
random.fit(x_treino, y_treino)

In [ ]:
previsoes_random = random.predict(x_teste)
previsoes_random

In [ ]:
y_teste

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
print(" Acurácia: %.2f%%" % (accuracy_score(y_teste, previsoes_random) * 100.0))

In [ ]:
pd.crosstab(le.inverse_transform(y_teste), le.inverse_transform(previsoes_random), rownames=['Atual'], colnames=['Previsto'], margins=True)

In [ ]:
print(classification_report(le.inverse_transform(y_teste), le.inverse_transform(previsoes_random)))

#### **Análise dados de treino**

In [ ]:
previsoes_treino = random.predict(x_treino)
previsoes_treino

In [ ]:
accuracy_score(y_treino, previsoes_treino)

In [ ]:
pd.crosstab(le.inverse_transform(y_treino), le.inverse_transform(previsoes_treino), rownames=['Atual'], colnames=['Previsto'], margins=True)

#### **Validação Cruzada**

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [ ]:
# Separando os dados em folds
kfold = KFold(n_splits = 30, shuffle=True, random_state = 5)

In [ ]:
# Criando o modelo
modelo = RandomForestClassifier(n_estimators=200, criterion='entropy', random_state = 0)
resultado = cross_val_score(modelo, previsores, alvo_cond, cv = kfold)

# Usamos a média e o desvio padrão
print("Acurácia Média: %.2f%%" % (resultado.mean() * 100.0))

## **XGBoost**

https://xgboost.readthedocs.io/en/stable/

In [ ]:
!pip install xgboost

In [ ]:
from xgboost import XGBClassifier 

In [ ]:
xg =  XGBClassifier(learning_rate=0.3, n_estimators=100, objective='multi:softmax', random_state=3)
xg.fit(x_treino, y_treino)

In [ ]:
previsoes_xg = xg.predict(x_teste)
previsoes_xg

In [ ]:
y_teste

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
print("Acurácia: %.2f%%" % (accuracy_score(y_teste, previsoes_xg) * 100.0))

In [ ]:
pd.crosstab(le.inverse_transform(y_teste), le.inverse_transform(previsoes_xg), rownames=['Atual'], colnames=['Previsto'], margins=True)

#### **Análise dados de treino**

In [ ]:
previsoes_treino = xg.predict(x_treino)
previsoes_treino

In [ ]:
accuracy_score(y_treino, previsoes_treino)

In [ ]:
pd.crosstab(le.inverse_transform(y_treino), le.inverse_transform(previsoes_treino), rownames=['Atual'], colnames=['Previsto'], margins=True)

### **Validação Cruzada**

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [ ]:
# Separando os dados em folds
kfold = KFold(n_splits = 30, shuffle=True, random_state = 5)

In [ ]:
# Criando o modelo
modelo = XGBClassifier(learning_rate=0.3, n_estimators=100, objective='multi:softmax', random_state=3)
resultado = cross_val_score(modelo, previsores, alvo_cond, cv = kfold)

# Usamos a média e o desvio padrão
print("Acurácia Média: %.2f%%" % (resultado.mean() * 100.0))

## **LightGBM**

In [ ]:
!pip install lightgbm

In [ ]:
import lightgbm as lgb 

In [ ]:
# Dataset para treino
dataset = lgb.Dataset(x_treino,label=y_treino)

In [ ]:
# Parâmetros
parametros = {'metric':'multi_logloss', # métrica para muitos atributos alvos
              'objective':'multiclass', # classificação múltipla classes     
              'learning_rate':0.03,
              'boosting_type': 'gbdt',
              'num_class': 19}

In [ ]:
# treinamento
lgbm=lgb.train(parametros,dataset, 200)


In [ ]:
previsoes_lgbm = lgbm.predict(x_teste)
previsoes_lgbm

In [ ]:
previsoes_lgbm.shape

In [ ]:
previsoes_lgbm = [np.argmax(line) for line in previsoes_lgbm] # retorna o índice correspondendo cada valor. Por exemplo: Região A é 0 a coluna, então todos os valores serão 0
previsoes_lgbm

In [ ]:
y_teste

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
print("Acurácia: %.2f%%" % (accuracy_score(y_teste, previsoes_lgbm) * 100.0))

#### **Análise dados de treino**

In [ ]:
previsoes_treino = lgbm.predict(x_treino)
previsoes_treino

In [ ]:
previsoes_treino = [np.argmax(line) for line in previsoes_treino] # retorna o índice correspondendo cada valor. Por exemplo: Região A é 0 a coluna, então todos os valores serão 0
previsoes_treino

In [ ]:
accuracy_score(y_treino, previsoes_treino)

In [ ]:
pd.crosstab(le.inverse_transform(y_treino), le.inverse_transform(previsoes_treino), rownames=['Atual'], colnames=['Previsto'], margins=True)

#### **Validação Cruzada**

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [ ]:
# Separando os dados em folds
kfold = KFold(n_splits = 30, shuffle=True, random_state = 5)

In [ ]:
# Criando o modelo
modelo = lgb.LGBMClassifier(metric = 'multi_logloss', objective = 'multiclass', boosting_type = 'gbdt', learning_rate = .03, num_class =19)
resultado = cross_val_score(modelo, previsores, alvo_cond, cv = kfold)

# Usamos a média e o desvio padrão
print("Acurácia Média: %.2f%%" % (resultado.mean() * 100.0))

## **CATBOOST**

https://catboost.ai/en/docs/

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
dataset_final_amostra

In [ ]:
previsores_cat = dataset_final_amostra.iloc[:, 4:8]
previsores_cat

In [ ]:
alvo_cat = dataset_final_amostra.iloc[:, 3]
alvo_cat_cond = le.transform(alvo_cat)
alvo_cat_cond

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# x: previsores e y: alvo
x_treino_cat, x_teste_cat, y_treino_cat, y_teste_cat = train_test_split(previsores_cat, alvo_cat_cond, test_size = 0.3, 
                                                           stratify = alvo_cat_cond, random_state = 0)

In [ ]:
catboost = CatBoostClassifier(task_type='CPU', iterations=200, learning_rate=0.03, random_state = 43,  loss_function='MultiClass',
                              eval_metric="Accuracy")

In [ ]:
catboost.fit(x_treino_cat, y_treino_cat, plot=True, eval_set=(x_teste, y_teste))

In [ ]:
previsoes_cat = catboost.predict(x_teste)
previsoes_cat

In [ ]:
y_teste

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
print("Acurácia: %.2f%%" % (accuracy_score(y_teste, previsoes_cat) * 100.0))

In [ ]:
pd.crosstab(le.inverse_transform(y_teste), le.inverse_transform(previsoes_cat), rownames=['Atual'], colnames=['Previsto'], margins=True)

In [ ]:
print(classification_report(le.inverse_transform(y_teste), le.inverse_transform(previsoes_cat)))

#### **Análise dados de treino**

In [ ]:
previsoes_treino = catboost.predict(x_treino)
previsoes_treino

In [ ]:
accuracy_score(y_treino, previsoes_treino)

In [ ]:
# Criando o modelo
modelo = CatBoostClassifier(task_type='CPU', iterations=200, learning_rate=0.4, random_state = 0, 
                              eval_metric="Accuracy")
resultado = cross_val_score(modelo, previsores, alvo_cond, cv = kfold)


In [ ]:
# Usamos a média e o desvio padrão
print("Acurácia Média: %.2f%%" % (resultado.mean() * 100.0))

## **Teste Definitivo** 

In [ ]:
dataset_final2_res

Extrair X, Y e Z

Para o modelo funcionar é necessário deixar os dados da latitude e longitude numa escala próxima a zero, ou seja, uma normalização e também representar em três dimensões com uso de seno e cosseno.

In [ ]:
import numpy as np
dataset_final2_res['x'] = np.cos(dataset_final2_res['Lat']) * np.cos(dataset_final2_res['Long'])
dataset_final2_res['y'] = np.cos(dataset_final2_res['Lat']) * np.sin(dataset_final2_res['Long'])
dataset_final2_res['z'] = np.sin(dataset_final2_res['Lat'])
dataset_final2_cluster = dataset_final2_res
dataset_final2_cluster

In [ ]:
dataset_cluster_regiao = dataset_final_amostra.loc[:, ["Regiao", "cluster"]].drop_duplicates(subset = ['Regiao'], keep='first')

In [ ]:
dataset_cluster_regiao

In [ ]:
# fazer o inner join para conseguir o cluster
dataset_final2_cluster = pd.merge(dataset_final2_res, dataset_cluster_regiao[['Regiao', 'cluster']], on = 'Regiao', how = 'left')

In [ ]:
dataset_final2_cluster 

In [ ]:
# dados da população restante
dataset_final2_cluster[["x", "y", "z"]].describe()

In [ ]:
# dados da amostra extraída
dataset_final_amostra[["x", "y", "z"]].describe()

In [ ]:
dataset_final2_cluster.dtypes

In [ ]:
previsores_def = dataset_final2_cluster.iloc[:, 4:8].values
previsores_def

In [ ]:
previsores_def.shape

In [ ]:
alvo_def = dataset_final2_cluster.iloc[:, 3].values
alvo_def

In [ ]:
alvo_def.shape

In [ ]:
le.fit(alvo_def)
alvo_cond_def = le.transform(alvo_def)
alvo_cond_def

 **Previsores e Alvos da População em Arquivos Disponíveis** 

In [ ]:

"""previsores_def = pd.read_csv(diretorio + '/Variaveis/previsores_populacao.csv',header=None)
previsores_def = previsores_def.values
previsores_def"""

In [ ]:
"""alvo_cond_def = pd.read_csv(diretorio + '/Variaveis/alvo_amostra.csv',header=None)
alvo_cond_def = alvo_cond_def.iloc[:, 0].values
alvo_cond_def"""

In [ ]:
previsoes_random_def = random.predict(previsores_def)
previsoes_random_def

In [ ]:
alvo_cond_def

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
print(" Acurácia: %.2f%%" % (accuracy_score(alvo_cond_def, previsoes_random_def) * 100.0))

In [ ]:
pd.crosstab(le.inverse_transform(alvo_cond_def), le.inverse_transform(previsoes_random_def), rownames=['Atual'], colnames=['Previsto'], margins=True)

In [ ]:
print(classification_report(le.inverse_transform(alvo_cond_def), le.inverse_transform(previsoes_random_def)))

## **Conclusão**


Baseado no arquivo previsores_amostra.csv e alvo_amostra.csv, o melhor algoritmo para ser utlizado no modelo proposto é o Random Forest:  
  
**SVM = 90.25% (teste) e 90.32% (validação cruzada)**  
**Árvore de Decisão = 99.78% (teste) e 99.83% (validação cruzada)**  
**Random Forest = 99.85% (teste) e 99.88% (validação cruzada)**  
**XGboost = 99.77% (teste) e 99.85% (validação cruzada)**  
**LightGBM = 99.72% (teste) e 99.75% (validação cruzada)**  
**CatBoost =  99.73% (teste) e 99.66% (validação cruzada)**

Escolhido o Random Forest, o algoritmo previu 97.10% corretamente no arquivo disponibilizado previsores_populacao.csv e alvo_populacao.csv